In [225]:
import sys
sys.path.append('../code/')
from utils.preprocess import read_json
import json
from collections import namedtuple
from wikidata.client import Client

In [226]:
gold_file = read_json("../datasets/qald9plus/qald_9_plus_test_wikidata.json")

In [227]:
prediction_file = read_json("../pred_files/mt5_linguistic/11_linguistic_mt5/en.json")

In [228]:
class Ques:
    def __init__(self, ques_dict) -> None:
        self.id = ques_dict["id"]
        self.question_string = ques_dict["question"][0]["string"]
        self.sparql = ques_dict["query"]["sparql"]
        try:
            self.answer = ques_dict["answers"][0]["results"]["bindings"]
        except:
            self.answer = []
        self.type = self.detect_query_type()
    
    def detect_query_type(self):
        return "entity"
    
    def print(self):
        print("id: " + self.id)
        print("question: \n" + self.question_string)
        print("sparql: \n" + self.sparql)
        print("answer:\n", self.answer)
    
    def print_along_with(self, ques):
        if self.id != ques.id:
            print("id of provided question differs from mine")
            return
        print("id: " + self.id)
        print("question: \n" + self.question_string)
        print("sparql: \n" + self.sparql)
        print("get sparql:  \n" + ques.sparql)
        print("answer: \n", self.answer)
        print("get answer: \n", ques.answer)

In [229]:
def get_wikidata_label(instance):
    client = Client()
    entity = client.get(instance, load=True)
    return entity.label

In [230]:
gold_ques_gen = (Ques(gold) for gold in gold_file["questions"])
pred_ques_gen = (Ques(pred) for pred in prediction_file["questions"])

# question, sparql query and answers

## id: 99

In [231]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 99
question: 
What is the time zone of Salt Lake City?
sparql: 
SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q23337>  <http://www.wikidata.org/prop/direct/P421>  ?o1 .  }
get sparql:  
SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q187893> <http://www.wikidata.org/prop/direct/P421> ?o1 . } 
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3134980'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2212'}}]
get answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5762'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

Wrong entity: Q187893(San Pedro de Atacama)

Q23337(Salt Lake City) is not in trianing data

## id: 98

In [232]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 98
question: 
Who killed Caesar?
sparql: 
SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q1048>  <http://www.wikidata.org/prop/direct/P157>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1203 wdt:P157 ?uri . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172248'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207370'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q294846'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1228715'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1243545'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q80462'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌
predicted entity: Q1203(John Lennon)

Q1048(Julius Caesar) is not in training data

## id: 86

In [233]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 86
question: 
What is the highest mountain in Germany?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P31 wd:Q8502 ; wdt:P2044 ?elevation ; wdt:P17 wd:Q183 . } ORDER BY DESC(?elevation) LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?res WHERE { ?res wdt:P31 wd:Q8502 . ?res wdt:P131+ wd:Q1202 . ?res wdt:P2044 ?height . } ORDER BY DESC(?height) LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3375'}}]
get answer: 
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q164561'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

A tuple is wrong, it gives the highest mountain in Q1202(Saxony), the rest parts are correct

It learns from 201 in training set: "What is the highest mountain in Saxony Germany?"

## id: 84

In [234]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 84
question: 
Which American presidents were in office during the Vietnam War?
sparql: 
SELECT DISTINCT ?o WHERE { <http://www.wikidata.org/entity/Q8740>  <http://www.wikidata.org/prop/direct/P580>  ?startWar . <http://www.wikidata.org/entity/Q8740>  <http://www.wikidata.org/prop/direct/P582>  ?endWar . <http://www.wikidata.org/entity/Q30>  <http://www.wikidata.org/prop/P6> ?blank . ?blank <http://www.wikidata.org/prop/statement/P6> ?o . ?blank <http://www.wikidata.org/prop/qualifier/P580> ?start . ?blank <http://www.wikidata.org/prop/qualifier/P582> ?end . FILTER (?startWar < ?start && ?endWar > ?end)} 
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q11817> <http://www.wikidata.org/prop/direct/P607> ?uri . }
answer: 
 [{'o': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9588'}}, {'o': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9640'}}, {'o': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9696'}}]
get answer: 
 [{'ur

absolutly no sense

out-of-scope: Q8740

training set 324: "Was U.S. president Jackson involved in a war?"

## id: 81

In [235]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 81
question: 
Butch Otter is the governor of which U.S. state?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?res WHERE { ?res wdt:P6 wd:Q39593 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1214 wdt:P6 ?uri . }
answer: 
 []
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6767775'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

it get the answer of "who is the governor of Q1214(Wyoming)"

Q39593 is not in the training data

training set 66: "Who is the governor of Wyoming?"

## id:66

In [236]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 66
question: 
Which artists were born on the same date as Rachel Stevens?
sparql: 
 SELECT ?s WHERE { <http://www.wikidata.org/entity/Q241665>  <http://www.wikidata.org/prop/direct/P569>  ?date . ?s <http://www.wikidata.org/prop/direct/P106> / <http://www.wikidata.org/prop/direct/P279>*   <http://www.wikidata.org/entity/Q483501> . ?s  <http://www.wikidata.org/prop/direct/P569>  ?date . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q241665 wdt:P569 ?date . BIND(YEAR(?date) as ?year) }
answer: 
 [{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q98538749'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9364870'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q95210650'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4717039'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3274687'}}, {'s': {'type': '

- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ❌

get the birth date of Rachel Stevens correctly, but miss the part of find other artists

out-of-scope: Q483501(artists)

## id:64

In [237]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 64
question: 
What is the profession of Frank Herbert?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q7934>  <http://www.wikidata.org/prop/direct/P106>  ?o1 .  }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q7934> <http://www.wikidata.org/prop/direct/P184> ?uri . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33231'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q36180'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1930187'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6625963'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18844224'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

out-of-scope: P106(occupation)

However, P194(doctoral advisor) is a relation which is plausible

training set 337: "Who was the doctoral supervisor of Albert Einstein?"

## id: 56

In [238]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 56
question: 
How many seats does the home stadium of FC Porto have?
sparql: 
SELECT DISTINCT ?num WHERE { <http://www.wikidata.org/entity/Q128446> <http://www.wikidata.org/prop/direct/P115>/<http://www.wikidata.org/prop/direct/P1083> ?num }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q51747 wdt:P1101 ?uri . }
answer: 
 [{'num': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '50431'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

floors above ground (P1101)

out-of-scope: FC Porto (Q128446)

FC Porto (Q128446) and maximum capacity (P1083) are not in training data

## id: 49

In [239]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 49
question: 
Which frequent flyer program has the most airlines?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> PREFIX ps: <http://www.wikidata.org/prop/statement/> SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer . }  GROUP BY ?frequentflyer ORDER BY DESC(COUNT(?airlines)) LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q46970 }
answer: 
 [{'frequentflyer': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q642970'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q69906'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q71988'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q73986'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

understand that answer should is related to airlines, but fail to understand "most" and "frequent flyer program"

out-of-scope: reward program (P4446)

training 58: "How many airlines are there?"|

## id: 44

In [240]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 44
question: 
Which European countries have a constitutional monarchy?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q458>  <http://www.wikidata.org/prop/direct/P150>  ?o1 . ?o1 <http://www.wikidata.org/prop/direct/P122>   <http://www.wikidata.org/entity/Q41614> }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P31 wd:Q7278 ; wdt:P17 wd:Q41 ; wdt:P1142 wd:Q3781399 . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q35'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q222897'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3567209'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikid

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: constitutional monarchy (Q41614)

## id: 42

In [241]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 42
question: 
Which countries have places with more than two caves?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?cave wdt:P31 wd:Q35509 ; wdt:P17 ?uri . } GROUP BY ?uri HAVING (COUNT(?cave) > 2)
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?lake wdt:P31 wd:Q23397 ; wdt:P17 wd:183 . ?uri wdt:P200 ?lake . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q236'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q242'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q241'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q252'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34020'}}, {'uri': {'type': 'uri', 

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 57: "Which rivers flow into a German lake?"

out-of-scope: cave (Q35509)

## id: 4

In [242]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 4
question: 
Which airports are located in California, USA?
sparql: 
SELECT DISTINCT ?s1 WHERE { ?s1  ?p1  <http://www.wikidata.org/entity/Q1248784> . ?s1  <http://www.wikidata.org/prop/direct/P276>  <http://www.wikidata.org/entity/Q99> .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q829 wdt:P113 ?uri . }
answer: 
 []
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: airport (Q1248784)

## id: 32

In [243]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 32
question: 
What is Angela Merkel’s birth name?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q567> <http://www.wikidata.org/prop/direct/P1477>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?name WHERE { wd:Q567 wdt:P1477 ?name . } 
answer: 
 [{'o1': {'xml:lang': 'de', 'type': 'literal', 'value': 'Angela Dorothea Kasner'}}]
get answer: 
 [{'name': {'xml:lang': 'de', 'type': 'literal', 'value': 'Angela Dorothea Kasner'}}]


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 151: "What is the birth name of Angela Merkel?"

## id: 31

In [244]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 31
question: 
Who is the mayor of Berlin?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q64>  <http://www.wikidata.org/prop/direct/P6>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q34370 wdt:P6 ?uri . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q102844'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q401107'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 300: "Who is the mayor of Rotterdam?"

## id: 29

In [245]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 29
question: 
Which countries in the European Union adopted the Euro?
sparql: 
 SELECT DISTINCT ?s1 WHERE { <http://www.wikidata.org/entity/Q458>  <http://www.wikidata.org/prop/direct/P150>  ?s1 .  ?s1  <http://www.wikidata.org/prop/direct/P38>  <http://www.wikidata.org/entity/Q4916> . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q6256> . ?uri <http://www.wikidata.org/prop/direct/P38> <http://www.wikidata.org/entity/Q4916> . }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q29'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q38'}}, {'s1': {'type': 'uri', 'value': 'http://www

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 102: "Which countries adopted the Euro?"

out-of-scope: contains the administrative territorial entity (P150)

## id: 68

In [246]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 68
question: 
Which country was Bill Gates born in?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q5284> <http://www.wikidata.org/prop/direct/P19>/<http://www.wikidata.org/prop/direct/P17> ?uri }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q9696 wdt:P19 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q30'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49142'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

out-of-scope: Bill Gates (Q5284)

## id: 22

In [247]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 22
question: 
How many grand-children did Jacques Cousteau have?
sparql: 
SELECT (COUNT(DISTINCT ?uri) as ?count) WHERE { <http://www.wikidata.org/entity/Q83233> <http://www.wikidata.org/prop/direct/P40>/<http://www.wikidata.org/prop/direct/P40> ?uri }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT (COUNT(DISTINCT ?uri) as ?c) WHERE { wd:Q43874 wdt:P40 ?uri . }
answer: 
 [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '4'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '8'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 144: "How many children does Eddie Murphy have?"

out-of-scope: Jacques Cousteau (Q83233)

doesn't understand P40/P40 for grand-children

## id: 214

In [248]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 214
question: 
Give me all professional skateboarders from Sweden.
sparql: 
 SELECT ?s1 WHERE { ?s1  <http://www.wikidata.org/prop/direct/P641> <http://www.wikidata.org/entity/Q842284> . ?s1  <http://www.wikidata.org/prop/direct/P19> ?o1 . ?o1 <http://www.wikidata.org/prop/direct/P131>* <http://www.wikidata.org/entity/Q34> . }
get sparql:  
SELECT ?s1 WHERE { ?s1 wdt:P106 <http://www.wikidata.org/entity/Q34> . ?s1 wdt:P31 <http://www.wikidata.org/entity/Q18127> . ?s1 wdt:P17 <http://www.wikidata.org/entity/Q16> . }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q959277'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3372856'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: sport (P641) skateboarding (Q842284)

## id: 210

In [249]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 210
question: 
Which monarchs of the United Kingdom were married to a German?
sparql: 
PREFIX p: <http://www.wikidata.org/prop/> PREFIX ps: <https://w3id.org/payswarm#> PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX bd: <http://www.bigdata.com/rdf#> PREFIX wikibase: <http://wikiba.se/ontology#> SELECT DISTINCT ?uri WHERE { { ?uri ?allowed wd:Q9134365 . VALUES ?allowed { wdt:P39 wdt:P97 } } UNION { ?uri wdt:P39 ?position . VALUES ?position { wd:Q116 wd:Q12097  } ?uri p:P39/pq:P642 ?kingdom . } VALUES ?kingdom { wd:Q161885 wd:Q174193 } . ?uri wdt:P26 ?spouse . ?spouse wdt:P19/wdt:P17 wd:Q183 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P106 wd:Q116 ; wdt:P26 ?spouse . ?spouse wdt:P27 wd:Q183 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 163: "Which monarchs were married to a German?"

out-of-scope: Great Britain (Q161885)

## id: 199

In [250]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 199
question: 
Give me all Argentine films.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri { ?uri wdt:P31 wd:Q11424 . ?uri wdt:P495 wd:Q414 }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?type wdt:P279* wd:Q11424 . ?uri wdt:P31 ?type ; wdt:P495 wd:Q35 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1537032'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1583599'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1593315'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1634612'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1659428'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1659455'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1675415'}}, {'uri': {'

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 51: Give me all Danish movies.

out-of-scope: Argentina (Q414)

## id: 203

In [251]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 203
question: 
How did Michael Jackson die?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q2831> <http://www.wikidata.org/prop/direct/P509> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q40912 wdt:P20 ?uri }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1033086'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12152'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q65'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: Michael Jackson (Q2831)

## id: 197

In [252]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 197
question: 
Which U.S. state has been admitted latest?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P571 ?date } ORDER BY DESC(?date) LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 ; wdt:P421 ?timezone . ?uri wdt:P31 wd:Q35657 ; wdt:P582 ?date . } ORDER BY DESC(?date) LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q782'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

understand "latest" with ORDER

## id: 196

In [253]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 196
question: 
Which classes does the Millepede belong to?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q11185669 wdt:P279 ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q7378 wdt:P279 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q167367'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110551885'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 255: "Which species does an elephant belong?"

out-of-scope: millipede (Q11185669)

## id: 188

In [254]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 188
question: 
Give me the homepage of Forbes.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q25161472 wdt:P856 ?uri } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q25161472 wdt:P98 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://forbes.com'}}, {'uri': {'type': 'uri', 'value': 'https://www.forbes.com'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q780842'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

training 220: "Who is the editor of Forbes?"

## id: 187

In [255]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 187
question: 
Who is the husband of Amanda Palmer?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P26 wd:Q240377 } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q7416 wdt:P40 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q210059'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q713468'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2449855'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 275: "List the children of Margaret Thatcher."

out-of-scope: Amanda Palmer (Q240377)

## id: 176

In [256]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 176
question: 
Who painted The Storm on the Sea of Galilee?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q2246489 wdt:P170 ?uri } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1163227 wdt:P50 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5598'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q210669'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 335: "Who wrote the book The Pillars of the Earth?"

out-of-scope: The Storm on the Sea of Galilee (Q2246489)

misinterpreted creator (P170) as author (P50)

## id: 62

In [257]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 62
question: 
What is the highest place of Karakoram?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q5469 wdt:P610 ?uri }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q35600> <http://www.wikidata.org/prop/direct/P610> ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q43512'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1142817'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 65: "What is the highest place of the Urals?"

out-of-scope: Karakoram (Q5469)

## id: 173

In [258]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 173
question: 
When did Finland join the EU?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX p: <http://www.wikidata.org/prop/> PREFIX ps: <http://www.wikidata.org/prop/statement/> SELECT ?date WHERE { wd:Q33 p:P463 ?membership . ?membership pq:P580 ?date . ?membership ps:P463 wd:Q458 . } 
get sparql:  
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q211> <http://www.wikidata.org/prop/P463> ?membership . ?membership <http://www.wikidata.org/prop/statement/P463> <http://www.wikidata.org/entity/Q458> . ?membership <http://www.wikidata.org/prop/qualifier/P580> ?date . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1995-01-01T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2004-05-01T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 138: "When did Latvia join the EU?"

out-of-scope: Finland (Q33)

## id: 168

In [259]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 168
question: 
Which actors play in Big Bang Theory?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q8539> <http://www.wikidata.org/prop/direct/P161> ?uri}
get sparql:  
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q858840> <http://www.wikidata.org/prop/direct/P161> ?o1 . } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q103939'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110154'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q120406'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q181900'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q190972'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q210200'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q229134'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q230335'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 412: "Give me all actors starring in Last Action Hero."

out-of-scope: The Big Bang Theory (Q8539)

## id: 166

In [260]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 166
question: 
Which computer scientist won an oscar?
sparql: 
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q82594> . ?uri <http://www.wikidata.org/prop/direct/P166> ?award . ?award <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q19020> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P1344 wd:Q127050 . ?scientist wdt:P279* wd:Q901 . ?uri wdt:P106 ?scientist . ?uri wdt:P166 wd:Q35637 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q93006'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q93161'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3022082'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5716583'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6388302'}}, {'uri': {'type

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 328: "Which scientist is known for the Manhattan Project and the Nobel Peace Prize?"

out-of-scope: computer scientist (Q82594) Academy Awards (Q19020)

## id: 160

In [261]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 160
question: 
Who wrote Harry Potter?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q8337> <http://www.wikidata.org/prop/direct/P50> ?uri}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q2695156 wdt:P50 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: Harry Potter (Q8337)

## id: 158

In [262]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


id: 158
question: 
Give me all writers that won the Nobel Prize in literature.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P106 wd:Q36180 ; wdt:P166 wd:Q37922 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P106 wd:Q36180 . ?uri wdt:P166 wd:Q37922 . } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q132701'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q132723'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q133042'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q134641'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q134644'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q159552'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q160478'}}, {'u

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 33: "Give me all writers that won the Nobel Prize in literature."

## id: 157

In [263]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 157
question: 
Give me English actors starring in Lovesick.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q12125895 wdt:P161 ?uri. ?uri wdt:P19 ?birthplace . ?birthplace wdt:P17 wd:Q145 . }
get sparql:  
SELECT ?s1 WHERE { ?s1 <http://www.wikidata.org/prop/direct/P161> <http://www.wikidata.org/entity/Q178552> . ?s1 <http://www.wikidata.org/prop/direct/P57> <http://www.wikidata.org/entity/Q192990> . ?s1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q11424> . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q103894'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q310343'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 389: "Which movies starring Mickey Rourke were directed by Guy Ritchie?"

out-of-scope: Lovesick (Q12125895)

## id: 150

In [264]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 150
question: 
In which city was the president of Montenegro born?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q236> <http://www.wikidata.org/prop/direct/P35>/<http://www.wikidata.org/prop/direct/P19> ?uri . ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q515> . }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q986> <http://www.wikidata.org/prop/direct/P35> ?uri}
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q200733'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q57348'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 398: "Who is the president of Eritrea?"

out-of-scope: Montenegro (Q236)

## id: 149

In [265]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 149
question: 
Which U.S. state has the highest population density?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?uri { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P1082 ?population . ?uri wdt:P2046 ?area . BIND((?population/?area) AS ?density ) } ORDER BY DESC(?density) LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 ; wdt:P2046 ?area ; wdt:P1082 ?population . BIND((?population/?area) AS ?density) . } ORDER BY DESC(?density) LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1408'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1408'}}]


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 16: "Which state of the USA has the highest population density?"

## id: 25

In [266]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 25
question: 
What is the longest river in China?
sparql: 
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q4022> . ?uri <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q148> . ?uri <http://www.wikidata.org/prop/direct/P2043> ?length } ORDER BY DESC(?length) LIMIT 1
get sparql:  
PREFIX psv: <http://www.wikidata.org/prop/statement/value/> PREFIX wikibase: <http://wikiba.se/ontology#> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?uri p:P2043
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5413'}}]
get answer: 
 []


- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

training 149: "What is the longest river?"

## id: 143

In [267]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 143
question: 
What is the area code of Berlin?
sparql: 
SELECT DISTINCT ?string WHERE { <http://www.wikidata.org/entity/Q64> <http://www.wikidata.org/prop/direct/P473> ?string . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q567 wdt:P69 ?uri . }
answer: 
 [{'string': {'type': 'literal', 'value': '030'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49738'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q154804'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q56230681'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q56230686'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: local dialing code (P473)

## id: 140

In [268]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 140
question: 
How many scientists graduated from an Ivy League university?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT (COUNT(DISTINCT ?uri) AS ?c) WHERE { ?uri wdt:P106 wd:Q901 . ?uri wdt:P69 ?university. wd:Q49113  wdt:P527 ?university . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT (COUNT(DISTINCT ?uri) AS ?c) WHERE { ?uri wdt:P106 wd:Q82955 ; wdt:P69 wd:Q49088 . }
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '522'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '751'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 140: "How many politicians graduated from Columbia University?"

out-of-scope: Ivy League (Q49113)

## id: 139

In [269]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 139
question: 
Which professional surfers were born in Australia?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT (COUNT(DISTINCT ?uri) AS ?c) WHERE { ?uri wdt:P19 ?birtplace . ?birtplace wdt:P17 wd:Q408 . ?uri wdt:P106 wd:Q13561328 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q10800557> . { ?uri <http://www.wikidata.org/prop/direct/P19> <http://www.wikidata.org/entity/Q183> . } UNION { ?uri <http://www.wikidata.org/prop/direct/P19>/<http://www.wikidata.org/prop/direct/
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '86'}}]
get answer: 
 []


- SPARQL syntax ❌
- entity ❌
- relation ✅
- answer ❌

training 82: "Which actors were born in Germany?"

out-of-scope: surfer (Q13561328)

## id: 137

In [270]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 137
question: 
Give me all Dutch parties.
sparql: 
 SELECT ?s1 WHERE { ?s1  <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q55> . ?s1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q7278> .  } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q163740 . { ?uri wdt:P159 wd:Q39 . } UNION { ?uri wdt:P159 ?city . ?city wdt:P31 wd:Q70208 } }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q59574'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q60172'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q117771'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q143058'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q231745'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q239333'}}, {'s1': {'type': 'uri', 'value':

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 42: "Give me all Swiss non-profit organizations."

## id: 136

In [271]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 136
question: 
How many moons does Mars have?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q111> <http://www.wikidata.org/prop/direct/P398> ?uri  . }
get sparql:  
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?mass WHERE { ?res wdt:P31 wd:Q61702557 . ?res wdt:P2067 ?mass . } ORDER BY ?mass LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7547'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7548'}}]
get answer: 
 [{'mass': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '1.5'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 156: "How heavy is Jupiter's lightest moon?"

out-of-scope: Mars (Q111) wdt:P398

## id: 134

In [272]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 134
question: 
What is Batman's real name?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q2695156> <http://www.wikidata.org/prop/direct/P460> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?name WHERE { wd:Q2695156 wdt:P1477 ?name . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3645503'}}]
get answer: 
 [{'name': {'xml:lang': 'en', 'type': 'literal', 'value': 'Bruce Wayne'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ✅

out-of-scope: said to be the same as (P460)

## id: 132

In [273]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 132
question: 
What is Elon Musk famous for?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q317521> <http://www.wikidata.org/prop/direct/P61> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q37079 wdt:P170 ?uri . }
answer: 
 []
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: Elon Musk (Q317521)

## id: 128

In [274]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 128
question: 
Who is the author of WikiLeaks?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q359>  <http://www.wikidata.org/prop/direct/P50>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q52 wdt:P112 ?uri . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q360'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q181'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q185'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 200: "Who created Wikipedia?"

out-of-scope: WikiLeaks (Q359)

## id: 124

In [275]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 124
question: 
When was Carlo Giuliani shot?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q164791>  <http://www.wikidata.org/prop/direct/P570>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q91 wdt:P20 ?uri } 
answer: 
 [{'o1': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2001-07-20T00:00:00Z'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4799733'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 7: "Where did Abraham Lincoln die?"

out-of-scope: Carlo Giuliani (Q164791)

## id: 122

In [276]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 122
question: 
Who was the wife of President Lincoln?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q91>  <http://www.wikidata.org/prop/direct/P26>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q91 wdt:P26 ?uri . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q215062'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q215062'}}]


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 254: "Who was the wife of U.S. president Lincoln?"

## id: 111

In [277]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 111
question: 
How many awards has Bertrand Russell?
sparql: 
 SELECT  (COUNT(DISTINCT ?o1) as ?count)  WHERE { <http://www.wikidata.org/entity/Q33760>  <http://www.wikidata.org/prop/direct/P166>  ?o1 . } 
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q319308> <http://www.wikidata.org/prop/direct/P166> ?uri . }
answer: 
 [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '7'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q572316'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q784589'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1316544'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2639529'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4865981'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q52382875'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/e

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: 
- Bertrand Russell (Q33760)

doesn't understand "How many" as "COUNT"

## id: 117

In [278]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 117
question: 
Is Pamela Anderson a vegan?
sparql: 
ASK where { <http://www.wikidata.org/entity/Q83325> <http://www.wikidata.org/prop/direct/P1576> <http://www.wikidata.org/entity/Q83364>}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> ASK WHERE { wd:Q7416 wdt:P106 wd:Q593644 . }
answer: 
 []
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 107: "Was Margaret Thatcher a chemist?"

out-of-scope: 
- Pamela Anderson (Q83325)
- lifestyle (P1576)
- vegetarianism (Q83364)

## id: 59

In [279]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 59
question: 
Which space probes were sent into orbit around the sun?
sparql: 
SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P522> <http://www.wikidata.org/entity/Q737809> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P1427 wd:Q177477 }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q152500'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q152800'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q156081'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q201953'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q224960'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q609059'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q653731'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q660356'}}, 

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 23: "Which spaceflights were launched from Baikonur?"

out-of-scope: 
- type of orbit (P522)
- heliocentric orbit (Q737809)

## id: 113

In [280]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 113
question: 
Which German cities have more than 250000 inhabitants?
sparql: 
 SELECT ?s1 WHERE { ?s1  <http://www.wikidata.org/prop/direct/P17>  <http://www.wikidata.org/entity/Q183> .  ?s1 <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q515> .  ?s1  <http://www.wikidata.org/prop/direct/P1082>  ?o1 . FILTER ( ?o1 > "250000"^^<http://www.w3.org/2001/XMLSchema#integer>)}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q515 ; wdt:P1082 ?population . FILTER(?population > 100000) }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q586'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1715'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1726'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1731'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2079'}

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

## id: 104

In [281]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 104
question: 
Who is the daughter of Robert Kennedy married to?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q25310> <http://www.wikidata.org/prop/direct/P40> ?daughter . ?daughter  <http://www.wikidata.org/prop/direct/P21> <http://www.wikidata.org/entity/Q6581072> . ?daughter  <http://www.wikidata.org/prop/direct/P26> ?uri .}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q43247 wdt:P40 ?child . ?child wdt:P26 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15133215'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q29938796'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q75326812'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q75326831'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q11673'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 27: 
"Who is the daughter of Ingrid Bergman married to?"

out-of-scope: 
- Robert F. Kennedy (Q25310)

## id: 10

In [282]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 10
question: 
How many students does the Free University of Amsterdam have?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q214341>  <http://www.wikidata.org/prop/direct/P2196>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT (COUNT(DISTINCT ?uri) AS ?c) WHERE { ?uri wdt:P106 wd:Q82955 ; wdt:P69 wd:Q49088 . }
answer: 
 [{'o1': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '31123'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '751'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 140: 
"How many politicians graduated from Columbia University?"

out-of-scope: 
- University of Amsterdam (Q214341)
- students count (P2196)

## id: 1

In [283]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 1
question: 
What is the revenue of IBM?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q37156>  <http://www.wikidata.org/prop/direct/P2139>  ?o1 .  }
get sparql:  
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?emloyees WHERE { wd:Q37156 wdt:P2295 ?emloyees . } 
answer: 
 [{'o1': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '79590000000'}}]
get answer: 
 [{'emloyees': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '11872000000'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

out-of-scope: 
- total revenue (P2139)

## id: 178

In [284]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 178
question: 
How many James Bond movies do exist?
sparql: 
 SELECT (COUNT(?s1) as ?c) WHERE { ?s1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q11424> . ?s1  ?p  <http://www.wikidata.org/entity/Q2009573> .  FILTER (?p = <http://www.wikidata.org/prop/direct/P144> || ?p = <http://www.wikidata.org/prop/direct/P674>) }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT (COUNT(DISTINCT ?uri) AS ?c) WHERE { ?uri wdt:P162 wd:Q72792 . }
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '17'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '594'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 50: 
"How many films did Hal Roach produce?"

out-of-scope: 
- based on (P144)
- characters (P674)

## id: 129

In [285]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 129
question: 
Who does the voice of Bart Simpson?
sparql: 
SELECT DISTINCT ?uri WHERE {  <http://www.wikidata.org/entity/Q5480> <http://www.wikidata.org/prop/direct/P725> ?uri }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q37079 wdt:P86 ?uri . }
answer: 
 []
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Bart Simpson (Q5480)

## id: 183

In [286]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 183
question: 
Who was Tom Hanks married to?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q2263>  <http://www.wikidata.org/prop/direct/P26>  ?o1 .  }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?o1 WHERE { wd:Q37079 wdt:P26 ?o1 . }
answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q234144'}}]
get answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174346'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233054'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37459'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been married to?"

## id: 181

In [287]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 181
question: 
Through which countries does the Yenisei river flow?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q78707> <http://www.wikidata.org/prop/direct/P17> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q5451 wdt:P17 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q159'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q668'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q836'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q837'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q843'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q889'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q917'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q148'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 81: 
"To which countries does the Himalayan mountain system extend?"

out-of-scope: 
- Yenisey (Q78707)

## id: 7

In [288]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 7
question: 
Give me all cars that are produced in Germany.
sparql: 
SELECT DISTINCT * WHERE { ?s1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q59773381> .  ?s1 <http://www.wikidata.org/prop/direct/P176> ?o1 . ?o1  <http://www.wikidata.org/prop/direct/P17>  <http://www.wikidata.org/entity/Q183> . ?o1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q786820> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?type wdt:P279* wd:Q728 . wd:Q155644 wdt:P287 ?designer . ?uri wdt:P279 ?type ; wdt:P287 ?designer . FILTER (?uri != wd:Q155644) }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q758718'}, 'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q23317'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q758729'}, 'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q23317'

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 14: 
"Which other weapons did the designer of the Uzi develop?"

out-of-scope: 
- automobile model series (Q59773381)
- automobile manufacturer (Q786820)


## id: 135

In [289]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 135
question: 
When did Michael Jackson die?
sparql: 
 SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q2831>  <http://www.wikidata.org/prop/direct/P570>  ?o1 .  } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q11806 wdt:P569 ?uri . }
answer: 
 [{'o1': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2009-06-25T00:00:00Z'}}]
get answer: 
 [{'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1735-10-30T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 252: 
"When was John Adams born?"

out-of-scope: 
- Michael Jackson (Q2831)

## id: 50

In [290]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 50
question: 
What is the highest volcano in Africa?
sparql: 
 SELECT DISTINCT ?s1 WHERE { ?s1  <http://www.wikidata.org/prop/direct/P30>  <http://www.wikidata.org/entity/Q15> . ?s1  <http://www.wikidata.org/prop/direct/P31>/<http://www.wikidata.org/prop/direct/P279>*  <http://www.wikidata.org/entity/Q8072> . ?s1 <http://www.wikidata.org/prop/direct/P2044> ?o1 . } ORDER BY DESC(?o1) LIMIT 1
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q8502> . ?uri <http://www.wikidata.org/prop/direct/P30> <http://www.wikidata.org/entity/Q3960> . ?uri <http://www.wikidata.org/prop/direct/P2044> ?height. } ORDER BY DESC(?height) LIMIT 1
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7296'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q178167'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 400: 
"What is the highest mountain in Australia?"

out-of-scope: 
- volcano (Q8072)

## id: 39

In [291]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 39
question: 
Which poet wrote the most books?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P106 wd:Q49757 . ?x wdt:P50 ?uri ; wdt:P31 wd:Q571 . } GROUP BY ?uri ORDER BY DESC(COUNT(?x)) LIMIT 1
get sparql:  
SELECT DISTINCT ?s1 WHERE { ?s1 <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q49757> . ?s2 <http://www.wikidata.org/prop/direct/P50> ?s1 . ?s2 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q571> . } GROUP BY ?s1 ORDER BY
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q198644'}}]
get answer: 
 []


- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ✅

predicted query from training 121: 
"Which poet wrote the most books?"

## id: 71

In [292]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 71
question: 
Give me all spacecrafts that flew to Mars.
sparql: 
SELECT ?s1 where  { ?s1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q389459> } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P1427 wd:Q177477 }
answer: 
 [{'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q48485'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176276'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q193538'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1770537'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3216558'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3445540'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q25415685'}}, {'s1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q87749354'}}, {'s1': {'type': 'uri', 'valu

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 23: 
"Which spaceflights were launched from Baikonur?"

out-of-scope: 
- Mars rover (Q389459)

## id: 213

In [293]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 213
question: 
Show me all Czech movies.
sparql: 
SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q11424> . { ?uri <http://www.wikidata.org/prop/direct/P495> <http://www.wikidata.org/entity/Q213> . } UNION { ?uri <http://www.wikidata.org/prop/direct/P495> <http://www.wikidata.org/entity/Q33946> . } }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?type wdt:P279* wd:Q11424 . ?uri wdt:P31 ?type ; wdt:P495 wd:Q35 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q755523'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q768103'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q772900'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q853705'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q861202'}}, {'uri': {'type': 'uri', 'valu

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 51: 
"Give me all Danish movies."

out-of-scope: 
- Czechoslovakia (Q33946)

## id: 209

In [294]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 209
question: 
Give me all taikonauts.
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P106 wd:Q11631 . ?uri wdt:P27 wd:Q148 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q11631> . { ?uri <http://www.wikidata.org/prop/direct/P27> <http://www.wikidata.org/entity/Q15180> } UNION { ?uri <http://www.wikidata.org/prop/direct/P27> <http://www.wikidata.org/entity/Q159> } }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q461840'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q470059'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q472683'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q557082'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q704554'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.o

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 41: 
"Give me all cosmonauts."

## id: 105

In [295]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 105
question: 
Which countries have more than ten volcanoes?
sparql: 
SELECT DISTINCT ?uri WHERE {  { ?volcano <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q8072> . } UNION { ?volcano <http://www.wikidata.org/prop/direct/P31> ?type . ?type <http://www.wikidata.org/prop/direct/P279>* <http://www.wikidata.org/entity/Q8072> . } ?volcano <http://www.wikidata.org/prop/direct/P17> ?uri . } GROUP BY ?uri HAVING(COUNT(?volcano)>10)
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?lake wdt:P31 wd:Q23397 ; wdt:P17 wd:183 . ?uri wdt:P200 ?lake . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q298'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q399'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q408'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q414'}}, {'

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 57: 
"Which rivers flow into a German lake?"

out-of-scope: 
- volcano (Q8072)

## id: 52

In [296]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 52
question: 
Give me all movies with Tom Cruise.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P161 wd:Q37079. }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P161 wd:Q37079. }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q108946'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110138'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110203'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110278'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q159054'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q188845'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q202028'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 171: 
"Give me all movies with Tom Cruise."

## id: 102

In [297]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 102
question: 
When did Dracula's creator die?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?date WHERE { wd:Q3266236 wdt:P170 ?author . ?author wdt:P570 ?date }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q2695156 wdt:P570 ?uri . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1912-04-20T00:00:00Z'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 210: 
"Who created Batman?"

out-of-scope: 
- Count Dracula (Q3266236)

## id: 21

In [298]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 21
question: 
Who created English Wikipedia?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q52 wdt:P170 ?uri . ?uri wdt:P31 wd:Q5 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q52 wdt:P112 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q181'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q185'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q181'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q185'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ✅

predicted query from training 200: 
"Who created Wikipedia?"

In [299]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 34
question: 
Give me all female German chancellors.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P21 wd:Q6581072 . ?uri wdt:P39 wd:Q4970706 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P39> <http://www.wikidata.org/entity/Q4970706> . } 
answer: 
 []
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q567'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2492'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2496'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2514'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2512'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2518'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2516'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2530'}}

## id: 145

In [300]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 145
question: 
Who owns Aldi?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { wd:Q125054 wdt:P355 ?uri }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q5930> <http://www.wikidata.org/prop/direct/P127> ?uri}
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15815751'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 365: 
"Who created Family Guy?"

out-of-scope: 
- ALDI (Q125054)
- has subsidiary (P355)

## id: 154

In [301]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 154
question: 
Which books were written by Danielle Steel?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P50 wd:Q72653 . ?uri wdt:P136 ?genre . ?genre wdt:P279* wd:Q571 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q571> . ?uri <http://www.wikidata.org/prop/direct/P50> <http://www.wikidata.org/entity/Q72653> . } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3030529'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3548978'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q643162'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q951140'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2416374'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2822694'}}, {'uri': {'type': 'uri', 'va

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 109: 
"Give me all books written by Danielle Steel."

## id: 198

In [302]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 198
question: 
Who was influenced by Socrates?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT * WHERE { ?uri wdt:P31 wd:Q5 . ?uri wdt:P737 wd:Q913 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q5582> <http://www.wikidata.org/prop/direct/P737> ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q81244'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q171303'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q179149'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q189506'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q312682'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q380190'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q470453'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q666230'}}, {'uri': {'t

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 339: 
"Who was Vincent van Gogh inspired by?"

## id: 162

In [303]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 162
question: 
How deep is Lake Chiemsee?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?n WHERE { wd:Q4138  wdt:P4511 ?n }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?num WHERE { wd:Q1207989 wdt:P2048 ?num . }
answer: 
 [{'n': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '72'}}]
get answer: 
 [{'num': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '101'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 304: 
"How high is the Yokohama Marine Tower?"

out-of-scope: 
- Chiemsee (Q4138)

## id: 212

In [304]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 212
question: 
Which companies work in the aerospace industry as well as in medicine?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE {  ?type wdt:P279* wd:Q4830453 . ?aerospace wdt:P279* wd:Q3477363 . ?healthcare wdt:P279* wd:Q15067276 . ?pharma wdt:P279* wd:Q507443 . ?uri wdt:P452 ?aerospace ; wdt:P452 ?healthcare . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?hovercraft wdt:P31/wdt:P279* wd:Q185757 ; wdt:P176 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q898208'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q218715'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q648280'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q682174'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.or

- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌

predicted query from training 253: 
"Which companies produce hovercrafts?"

out-of-scope: 
- aerospace industry (Q3477363)
- health care industry (Q15067276)

## id: 96

In [305]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 96
question: 
Which professional surfers were born on the Philippines?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT * WHERE { ?uri wdt:P19 ?birtplace . ?birtplace wdt:P17 wd:Q928 . ?uri wdt:P106  wd:Q13561328 . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q10800557> . { ?uri <http://www.wikidata.org/prop/direct/P19> <http://www.wikidata.org/entity/Q183> . } UNION { ?uri <http://www.wikidata.org/prop/direct/P19>/<http://www.wikidata.org/prop/direct/
answer: 
 []
get answer: 
 []


- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

predicted query from training 82: 
"Which actors were born in Germany?"

out-of-scope: 
- surfer (Q13561328)

## id: 40

In [306]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 40
question: 
Who were the parents of Queen Victoria?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?uri WHERE { wd:Q9439 ?childof ?uri . VALUES ?childof { wdt:P22 wdt:P25 }}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q154946 wdt:P40 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q157009'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q58067'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q263623'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q268821'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q435324'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q29574'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Victoria (Q9439)

## id: 79

In [307]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 79
question: 
Are there any castles in the United States?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> ASK WHERE { { ?uri wdt:P31 wd:Q23413. } UNION { ?uri wdt:P31 ?castlesubtype . ?castlesubtype wdt:P279* wd:Q23413 } ?uri wdt:P17 wd:Q30 }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q8502 ; wdt:P17 wd:Q408 ; wdt:P1082 ?population . } ORDER BY DESC(?population) LIMIT 1
answer: 
 []
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6919413'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- castle (Q23413)

## id: 141

In [308]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 141
question: 
Which languages are spoken in Estonia?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . }
answer: 
 [{'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7737'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9072'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32762'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34017'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3196221'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12361545'}}]
get answer: 
 [{'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7737'}}, {'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q907

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 187: 
"Which languages are spoken in Estonia?"

## id: 80

In [309]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 80
question: 
Give me a list of all critically endangered birds.
sparql: 
SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P171>* <http://www.wikidata.org/entity/Q5113> . ?uri <http://www.wikidata.org/prop/direct/P141> <http://www.wikidata.org/entity/Q219127> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P279* wd:Q38280 FILTER(?uri != wd:Q38280) }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27074577'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q212526'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q657365'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q852146'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17517273'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1265085'}}, {'uri': {'type': 'uri', 'value': 'http:

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 185: 
"Give me all breeds of the German Shepherd dog."

out-of-scope: 
- birds (Q5113)
- IUCN conservation status (P141)
- Critically Endangered (Q219127)

## id: 159

In [310]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 159
question: 
Which countries are connected by the Rhine?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri ?x ?y  WHERE { wd:Q584  wdt:P17 ?uri . ?uri wdt:P31 wd:Q6256 . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q794 wdt:P47 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q347'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q39'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q40'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q142'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q183'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q131083'}}, {'uri': {'type': 'uri', 'value': 'htt

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 262: 
"With how many countries Iran has borders?"

out-of-scope: 
- Rhine (Q584)

## id: 107

In [311]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 107
question: 
Who was the father of Queen Elizabeth II?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?uri WHERE { wd:Q9682 wdt:P22  ?uri . }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q989> <http://www.wikidata.org/prop/P39> ?position . ?position <http://www.wikidata.org/prop/statement/P39> <http://www.wikidata.org/entity/Q19546> . ?position <http://www.wikidata.org/prop/qualifier/P1366> ?uri } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q280856'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2494'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 72: 
"Which pope succeeded John Paul II?"

out-of-scope: 
- Elizabeth II (Q9682)

## id: 163

In [312]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 163
question: 
Give me all chemical elements.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q11344 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P279 wd:Q373822 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q556'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q560'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q568'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q569'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q618'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q623'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q627'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q629'}}, {'uri': {'type': '

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 103: 
"Give me all types of eating disorders."

out-of-scope: 
- chemical element (Q11344)

## id: 211

In [313]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 211
question: 
Give me all American presidents of the last 20 years.
sparql: 
PREFIX ps: <http://www.wikidata.org/prop/statement/> PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> SELECT DISTINCT ?uri WHERE { ?uri p:P39 ?position . ?position ps:P39 wd:Q11696 . ?position pq:P582 ?termEnd. FILTER((YEAR(NOW()) - YEAR(?termEnd)) <= 20) }
get sparql:  
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P39 wd:Q9259 ; wdt:P575 ?date . BIND(xsd:integer(YEAR(?date)) as ?year) . BIND(xsd:
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96181216'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1124'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q22686'}}, {'uri': {'type': 'uri', 'value': 'http://www.wik

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

predicted query from training 68: 
"Give me all world heritage sites designated within the **past two years**."

## id: 51

In [314]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 51
question: 
When is the movie Worst Case Scenario going to be in cinemas in the Netherlands?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?date WHERE { wd:Q6667101 wdt:P577 ?date }
get sparql:  
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q8641370> <http://www.wikidata.org/prop/direct/P580> ?date . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2008-01-30T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1944-06-06T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 356: 
"When did Operation Overlord commence?"

out-of-scope: 
- Worst Case Scenario (Q6667101)

## id: 144

In [315]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 144
question: 
Whom did Lance Bass marry?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q443120> <http://www.wikidata.org/prop/direct/P26> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?o1 WHERE { wd:Q37079 wdt:P26 ?o1 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19668300'}}]
get answer: 
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174346'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233054'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37459'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been married to?"

out-of-scope: 
- Lance Bass (Q443120)

## id: 131

In [316]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 131
question: 
What languages are spoken in Pakistan?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT * { wd:Q843 wdt:P2936 ?uri . }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q739> <http://www.wikidata.org/prop/direct/P37> ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1617'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1860'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32288'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32480'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32900'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33028'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33049'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33086'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikid

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Pakistan (Q843)

## id: 120

In [317]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 120
question: 
Who is the daughter of Bill Clinton married to?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT * { wd:Q1124 wdt:P40 ?child . ?child wdt:P26 ?uri }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q37079 wdt:P26 ?uri . }
answer: 
 [{'child': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q229671'}, 'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15142858'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174346'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233054'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37459'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been **married to**?"

out-of-scope: 
- Bill Clinton (Q1124)

## id: 164

In [318]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 164
question: 
Who was on the Apollo 11 mission?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q43653> <http://www.wikidata.org/prop/direct/P1029> ?uri}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q178871 wdt:P1029 ?uri }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q104859'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1615'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2252'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174979'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q295873'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q375065'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 20: 
"Give me the Apollo 14 astronauts."

out-of-scope: 
- Apollo 11 (Q43653)

## id: 151

In [319]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 151
question: 
Give me all B-sides of the Ramones.
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX ps: <http://www.wikidata.org/prop/statement/> PREFIX p: <http://www.wikidata.org/prop/> SELECT ?res WHERE { ?res wdt:P175 wd:Q483407 . ?res p:P1433 _:a ._:a pq:P2868 wd:Q13432985 . }
get sparql:  
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX ps: <http://www.wikidata.org/prop/statement/> PREFIX p: <http://www.wikidata.org/prop/> SELECT ?res WHERE { ?res wdt:P175 wd:Q483407 . ?res p:P1433 _:
answer: 
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q91353165'}}]
get answer: 
 []


- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ❌

predicted query from training 175: 
"Give me all B-sides of the Ramones"

## id: 63

In [320]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 63
question: 
Who was called Scarface?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri { ?uri wdt:P1449 ?nickname . FILTER REGEX(STR(?nickname), "^Scarface$") }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q5930> <http://www.wikidata.org/prop/direct/P170> ?uri}
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q80048'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q629034'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q188492'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 365: 
"Who created Family Guy?"

out-of-scope: 
- nickname (P1449)

## id: 27

In [321]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 27
question: 
Which rivers flow into the North Sea?
sparql: 
SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q4022> . ?uri <http://www.wikidata.org/prop/direct/P403> <http://www.wikidata.org/entity/Q1693> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?lake wdt:P31 wd:Q23397 ; wdt:P17 wd:183 . ?uri wdt:P200 ?lake . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q216373'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q217061'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q221807'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q300661'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q338299'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q366284'}}, {'uri': {'type': 'uri',

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 57: 
"**Which rivers flow into** a German lake?"

out-of-scope: 
- North Sea (Q1693)

## id: 156

In [322]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 156
question: 
Where is Fort Knox located?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri { wd:Q8962067 wdt:P131+ ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?res WHERE { wd:Q983650 wdt:P131+ ?res . ?res wdt:P31 wd:Q35657 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q491878'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q505438'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1603'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q30'}}]
get answer: 
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1603'}}]


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ❌✅

predicted query from training 172: 
"In which U.S. state is Fort Knox located?"

## id: 207

In [323]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 207
question: 
Which daughters of British earls died at the same place they were born at?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri { ?uri wdt:P19 ?birthplace . ?uri wdt:P20 ?deathplace . FILTER(?birthplace = ?deathplace) ?uri wdt:P21 wd:Q6581072 . ?uri wdt:P22 ?father . ?father wdt:P97/wdt:P460 wd:Q1128240. ?father wdt:P27*/wdt:P1366*/wdt:P706* wd:Q38272 . }
get sparql:  
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P19 _:a . ?uri wdt:P20 _:a . ?uri wdt:P106/wdt:P279* wd:Q10873124 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1388169'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7422829'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q250179'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q254159'

- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌

predicted query from training 169: 
"Which Chess players **died in the same place they were born in?**"

out-of-scope: 
- British Isles (Q38272)

## id: 24

In [324]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 24
question: 
How many emperors did China have?
sparql: 
SELECT (COUNT(DISTINCT ?uri) as ?count) WHERE {  ?uri <http://www.wikidata.org/prop/direct/P39> <http://www.wikidata.org/entity/Q268218>}
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P39> <http://www.wikidata.org/entity/Q4970706> . } 
answer: 
 [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '356'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q567'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2492'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2496'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2514'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2512'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2518'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2516'}}, {'uri': {'

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 24: 
"Give me all **federal chancellors** of Germany."

out-of-scope: 
- Emperor of China (Q268218)

## id: 133

In [325]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 133
question: 
What are the names of the Teenage Mutant Ninja Turtles?
sparql: 
SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P361> <http://www.wikidata.org/entity/Q1381762> . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q2695156 wdt:P170 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q323920'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q323925'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q323934'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2078291'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q313048'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q464282'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 210: 
"Who created Batman?"

out-of-scope: 
- Teenage Mutant Ninja Turtles (Q1381762)

## id: 103

In [326]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 103
question: 
Where does Piccadilly start?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q1124023> <http://www.wikidata.org/prop/direct/P559> ?uri}
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1339 wdt:P131 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q215255'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2166146'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

Where: located in the administrative territorial entity (P131)

out-of-scope: 
- Piccadilly (Q1124023)

## id: 165

In [327]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 165
question: 
What is the name of the university where Obama's wife studied?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri ?institute WHERE { wd:Q76 wdt:P26/wdt:P69/wdt:P361* ?uri . ?uri wdt:P31*/wdt:P279* wd:Q3918 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q2685 wdt:P69 ?uri . ?uri wdt:P31/wdt:P279* wd:Q3918 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13371'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21578'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174710'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q10943718'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55044'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 266: 
"Did Arnold Schwarzenegger attend a university?",
but change "ASK" to "SELECT"

## id: 108

In [328]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 108
question: 
When did Paraguay proclaim its independence?
sparql: 
SELECT ?o1 WHERE { <http://www.wikidata.org/entity/Q733>  <http://www.wikidata.org/prop/direct/P571>  ?o1 .  }
get sparql:  
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q9903> <http://www.wikidata.org/prop/direct/P576> ?date . }
answer: 
 [{'o1': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1811-01-01T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1644-01-01T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 368: 
"When did the Ming dynasty **dissolve**?"

out-of-scope: 
- Paraguay (Q733)

## id: 15

In [329]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 15
question: 
How short is the shortest active NBA player?
sparql: 
SELECT DISTINCT ?height WHERE { ?uri <http://www.wikidata.org/prop/direct/P106>  <http://www.wikidata.org/entity/Q3665646> . ?uri <http://www.wikidata.org/prop/direct/P54>/<http://www.wikidata.org/prop/direct/P118> <http://www.wikidata.org/entity/Q155223> .  ?uri <http://www.wikidata.org/prop/P54> ?teamMembership . FILTER NOT EXISTS { ?teamMembership <http://www.wikidata.org/prop/qualifier/P582> ?endDate } ?uri  <http://www.wikidata.org/prop/direct/P2048> ?height } ORDER BY ?height LIMIT 1
get sparql:  
PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX wikibase: <http://wikiba.se/ontology#> PREFIX psv: <http://www.wikidata.org/prop/statement/value/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> SELECT DISTINCT ?uri WHERE { ?uri w
answer: 
 [{'height': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type':

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- National Basketball Association (Q155223)

## id: 45

In [330]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 45
question: 
Where did Abraham Lincoln die?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q91 wdt:P20 ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q91 wdt:P20 ?uri } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4799733'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4799733'}}]


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 7: 
"Where did Abraham Lincoln die?"

## id: 82

In [331]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 82
question: 
When was Jack Wolfskin founded?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?date WHERE { wd:Q536133 wdt:P571 ?date . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?date WHERE { wd:Q622188 wdt:P571 ?date . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1981-01-01T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1948-03-17T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 92: 
"When were the Hells Angels **founded**?"

out-of-scope: 
- Jack Wolfskin (Q536133)

## id: 9

In [332]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 9
question: 
In which city is the headquarter of Air China?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q51747 wdt:P159 ?uri } 
get sparql:  
SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q5089> <http://www.wikidata.org/prop/direct/P36> ?o1 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q393482'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q956'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

## id: 201

In [333]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 201
question: 
What is the founding year of the brewery that produces Pilsner Urquell?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?date WHERE { wd:Q948831 wdt:P571 ?date . } 
get sparql:  
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q243278> <http://www.wikidata.org/prop/direct/P527> ?date . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1842-01-01T00:00:00Z'}}, {'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1992-01-01T00:00:00Z'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Plzeňský Prazdroj (Q948831)

## id: 171

In [334]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 171
question: 
When did the Boston Tea Party take place?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q19024> <http://www.wikidata.org/prop/direct/P585> ?uri}
get sparql:  
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q8641370> <http://www.wikidata.org/prop/direct/P580> ?date . }
answer: 
 [{'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1773-12-16T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1944-06-06T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 356: 
"When did Operation Overlord commence?"

out-of-scope: 
- Boston Tea Party (Q19024)

## id: 182

In [335]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 182
question: 
Which animals are critically endangered?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P141 wd:Q219127 }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P279* wd:Q38280 FILTER(?uri != wd:Q38280) }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q72105'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q74376'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q74625'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q76411'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q81660'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q95036'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q113654'}}, {'uri': {'type': 'uri', 'value': 'http://ww

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 185: 
"Give me all breeds of the German Shepherd dog."

out-of-scope: 
- IUCN conservation status (P141)
- Critically Endangered (Q219127)

## id: 13

In [336]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 13
question: 
Which politicians were married to a German?
sparql: 
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q82955> . ?uri <http://www.wikidata.org/prop/direct/P26> ?spouse . { ?spouse <http://www.wikidata.org/prop/direct/P27> <http://www.wikidata.org/entity/Q183> . } UNION { ?spouse <http://www.wikidata.org/prop/direct/P19>/<http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q183> } } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P106 wd:Q116 ; wdt:P26 ?spouse . ?spouse wdt:P27 wd:Q183 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5879'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7231'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q8442'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9387'

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ❌✅

predicted query from training 17: 
"Which monarchs were married to a German?"

## id: 114

In [337]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 114
question: 
How big is the earth's diameter?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?c WHERE { wd:Q2 wdt:P2386 ?c. }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?mass WHERE { ?res wdt:P31 wd:Q61702557 . ?res wdt:P2067 ?mass . } ORDER BY ?mass LIMIT 1
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '12742'}}]
get answer: 
 [{'mass': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '1.5'}}]


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 156: 
"How heavy is Jupiter's lightest moon?"

out-of-scope: 
- diameter (P2386)

## id: 92

In [338]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 92
question: 
Is the wife of President Obama called Michelle?
sparql: 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> ASK WHERE { wd:Q76 wdt:P26 ?obj . ?obj rdfs:label ?label FILTER( REGEX( ?label, "Michelle" ) ) }
get sparql:  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> ASK WHERE { wd:Q76 wdt:P26 ?wife . ?wife rdfs:label ?name . FILTER regex(?name, "Michelle") }
answer: 
 []
get answer: 
 []


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 227: 
"Is the wife of president Obama called Michelle?"

## id: 194

In [339]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 194
question: 
Which U.S. state has the abbreviation MN?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P1813 ?name . FILTER( REGEX( ?name, "^MN$")) } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?res WHERE { wd:Q983650 wdt:P131+ ?res . ?res wdt:P31 wd:Q35657 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1527'}}]
get answer: 
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1603'}}]


- SPARQL syntax ✅
- entity ❌✅
- relation ❌✅
- answer ❌

predicted query from training 172: 
"In which **U.S. state** is Fort Knox located?"

out-of-scope: 
- short name (P1813)

## id: 175

In [340]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 175
question: 
When did Muhammad die?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?date WHERE { wd:Q9458 wdt:P570 ?date . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q692 wdt:P570 ?uri . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '0632-06-11T00:00:00Z'}}]
get answer: 
 [{'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1616-05-03T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 239: 
"When was the death of Shakespeare?"

out-of-scope: 
- Muhammad (Q9458)

## id: 60

In [341]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 60
question: 
Who is the governor of Texas?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1439 wdt:P1313 wd:Q5589725 . wd:Q5589725 wdt:P1308 ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1214 wdt:P6 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q215057'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q216547'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q237918'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q285261'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q311293'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q402676'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q449664'}}, {'uri': {'type': 'uri', 'value': 'http:/

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 66: 
"**Who is the governor of** Wyoming?"

but relations are not the same

out-of-scope: 
- Texas (Q1439)

## id: 110

In [342]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 110
question: 
Which movies did Kurosawa direct?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT * WHERE { ?uri wdt:P57 wd:Q8006 . } 
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P57> <http://www.wikidata.org/entity/Q2001>}
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q135465'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q152105'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q189540'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q244878'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q257818'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q418809'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q565231'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q575014'}}, {'uri': {'type': 'uri', 'value':

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 282: 
"Which films did Stanley Kubrick direct?"

out-of-scope: 
- Akira Kurosawa (Q8006)

## id: 206

In [343]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 206
question: 
What was the name of the famous battle in 1836 in San Antonio?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P276 wd:Q975 . ?uri wdt:P31 wd:Q178561 . ?uri wdt:P585 ?time . FILTER( YEAR(?time) = 1836 ) } 
get sparql:  
SELECT DISTINCT ?name WHERE { <http://www.wikidata.org/entity/Q128160> <http://www.wikidata.org/prop/direct/P570> ?date . FILTER(year(?date)=1945) }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q235344'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- San Antonio (Q975)
- battle (Q178561)

## id: 97

In [344]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 97
question: 
Give me the official websites of actors of the television show Charmed.
sparql: 
PREFIX p: <http://www.wikidata.org/prop/> PREFIX ps: <http://www.wikidata.org/prop/statement/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q162371 p:P161/ps:P161/wdt:P856 ?uri . }
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q162371> <http://www.wikidata.org/prop/direct/P161>/<http://www.wikidata.org/prop/direct/P569> ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.alyssa.com'}}, {'uri': {'type': 'uri', 'value': 'https://www.rosemcgowan.com/'}}, {'uri': {'type': 'uri', 'value': 'http://www.shannen-doherty.net'}}, {'uri': {'type': 'uri', 'value': 'http://www.tedking.com/'}}]
get answer: 
 [{'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1971-04-12T00:00:00Z'}}, {'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 141: 
"Give me the birthdays of all actors of the television show Charmed."

## id: 169

In [345]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 169
question: 
Give me all libraries established before 1400.
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT * WHERE { ?uri wdt:P31/wdt:P279* wd:Q7075 . ?uri wdt:P571 ?date . FILTER( YEAR(?date) < 1400 ) } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q7075 ; wdt:P571 ?date . FILTER (YEAR(?date) < 1400 ) }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2326916'}, 'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1049-01-01T00:00:00Z'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q24101'}, 'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1364-01-01T00:00:00Z'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q667615'}, 'date': {'datatype': 'http://www.w

- SPARQL syntax ✅
- entity ✅
- relation ✅❌
- answer ❌

predicted query from training 113: 
"Give me all libraries established earlier than 1400."

## id: 38

In [346]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 38
question: 
Give me all Frisian islands that belong to the Netherlands.
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { { wd:Q76936 wdt:P527+ ?uri . } UNION { wd:Q76936 wdt:P527* ?islandgroup . ?uri wdt:P361+ ?islandgroup . } ?uri wdt:P17 wd:Q55 . ?uri wdt:P31 wd:Q23442 . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q23397 ; wdt:P17 wd:Q35 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9966'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207851'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1342240'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1479319'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1546490'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/ent

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 143: 
"Give me a list of all lakes in Denmark."

out-of-scope: 
- Frisian Islands (Q76936)

## id: 192

In [347]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 192
question: 
Which museum exhibits The Scream by Munch?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q471379> <http://www.wikidata.org/prop/direct/P527>/<http://www.wikidata.org/prop/direct/P276> ?uri . ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q207694>}
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q33506> <http://www.wikidata.org/prop/direct/P276> ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q844926'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1132918'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3330707'}}]
get answer: 
 []


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- The Scream (Q471379)
- art museum (Q207694)

## id: 78

In [348]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 78
question: 
When was the De Beers company founded?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?date WHERE { wd:Q471891 wdt:P571 ?date . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?date WHERE { wd:Q622188 wdt:P571 ?date . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1888-01-01T00:00:00Z'}}]
get answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1948-03-17T00:00:00Z'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 92: 
"When were the Hells Angels founded?"

out-of-scope: 
- De Beers (Q471891)

## id: 19

In [349]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 19
question: 
Who became president after JFK died?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q9696> <http://www.wikidata.org/prop/P39> ?position . ?position  <http://www.wikidata.org/prop/statement/P39> <http://www.wikidata.org/entity/Q11696> . ?position <http://www.wikidata.org/prop/qualifier/P1366> ?uri . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q9696 wdt:P20 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9640'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3896131'}}]


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 77: 
"Where was JFK assassinated?"

## id: 190

In [350]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 190
question: 
Who are the parents of the wife of Juan Carlos I?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q19943 wdt:P26 ?spouse . ?uri wdt:P40 ?spouse . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1203 wdt:P40 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q155394'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q61351'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q311238'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q357974'}}]


- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌
  
out-of-scope: 
- Juan Carlos I of Spain (Q19943)


In [351]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 174
question: 
Who is the novelist of the work a song of ice and fire?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q45875 ?p ?uri . ?uri wdt:P106 wd:Q6625963 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q726382 wdt:P50 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q181677'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9215'}}]


In [352]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 189
question: 
In which studio did the Beatles record their first album?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?album wdt:P31 wd:Q482994 ; wdt:P175 wd:Q1299 ; wdt:P577 ?date ; wdt:P483 ?uri . } ORDER BY ?date LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?label WHERE { wd:Q59150527 wdt:P527 ?uri . ?uri wdt:P577 ?date . ?uri wdt:P264 ?label . } ORDER BY ?date LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q209651'}}]
get answer: 
 [{'label': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q202585'}}]


In [353]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 152
question: 
Which beer brewing companies are located in North-Rhine Westphalia?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q131734 . ?uri wdt:P159*/wdt:P31*/wdt:P131 wd:Q1198 . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q131734 . ?uri wdt:P17 wd:Q408 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q152143'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q900372'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q900382'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q900425'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q900434'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q152169'}}, {'uri': {'type': 'uri', 'value': 'http://ww

In [354]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 94
question: 
When did princess Diana die?
sparql: 
SELECT DISTINCT ?date WHERE { <http://www.wikidata.org/entity/Q9685> <http://www.wikidata.org/prop/direct/P570> ?date  . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q692 wdt:P570 ?uri . }
answer: 
 [{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1997-08-31T00:00:00Z'}}]
get answer: 
 [{'uri': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1616-05-03T00:00:00Z'}}]


In [355]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 8
question: 
Who founded Intel?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q248 wdt:P112 ?uri . }  
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q49740 wdt:P178 ?uri . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q241735'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q243969'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q432929'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1129295'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2283'}}]


In [356]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 95
question: 
Who is the youngest player in the Premier League?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?team wdt:P118 wd:Q9448 . ?uri wdt:P54 ?team ; wdt:P569 ?dateOfBirth . } ORDER BY DESC(?dateOfBirth) LIMIT 1
get sparql:  
PREFIX pq: <http://www.wikidata.org/prop/qualifier/> PREFIX wikibase: <http://wikiba.se/ontology#> PREFIX psv: <http://www.wikidata.org/prop/statement/value/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> SELECT DISTINCT ?uri WHERE { ?uri w
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q105104385'}}]
get answer: 
 []


In [357]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 119
question: 
Which instruments does Cat Stevens play?
sparql: 
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q154216> <http://www.wikidata.org/prop/direct/P1303> ?uri . } 
get sparql:  
SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q154216> <http://www.wikidata.org/prop/direct/P1303> ?uri . } 
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17172850'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5994'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6607'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q46185'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17172850'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5994'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6607'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q46185'}}]


In [358]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 115
question: 
How many rivers and lakes are in South Carolina?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT (COUNT(?uri) AS ?c) WHERE { ?uri wdt:P131 wd:Q1456 . ?uri wdt:P31/wdt:P279* wd:Q23397 . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT (COUNT(?uri) AS ?c) WHERE { ?uri wdt:P31 wd:Q4022 . ?lake wdt:P31 wd:Q23397 ; wdt:P17 wd:183 . ?uri wdt:P200 ?lake . }
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '22'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '0'}}]


In [359]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 116
question: 
Who was called Rodzilla?
sparql: 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#> SELECT * WHERE { ?uri skos:altLabel "Rodzilla"@en . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q23215 wdt:P170 ?uri }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q201608'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3438213'}}]
get answer: 
 []


In [360]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 138
question: 
Give me the capitals of all countries in Africa.
sparql: 
SELECT DISTINCT ?uri WHERE { ?country <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q6256> . ?country  <http://www.wikidata.org/prop/direct/P30> <http://www.wikidata.org/entity/Q15> . ?country <http://www.wikidata.org/prop/direct/P36> ?uri . }
get sparql:  
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q6256> . ?uri <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q408> . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3561'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3624'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3659'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3761'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3870'}}, {'uri': {'type': 'uri', 'value': 'http://www.

In [361]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 177
question: 
Which bridges are of the same type as the Manhattan Bridge?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri { wd:Q125050 wdt:P31 ?type . ?uri wdt:P31 ?type . ?uri wdt:P31/wdt:P279* wd:Q12280 . FILTER( ?type != wd:Q12280 ) }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q12280 ; wdt:P177 wd:Q1471 . }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q162459'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q259220'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q328874'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q436044'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q505745'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q509824'}}, {'uri': {'type': 'ur

In [362]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 101
question: 
How many companies were founded by the founder of Facebook?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ( COUNT( DISTINCT ?uri ) AS ?c ) WHERE { ?uri wdt:P112 wd:Q36215 . ?uri wdt:P31/wdt:P279* wd:Q783794 . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT (COUNT(DISTINCT ?uri) as ?c) WHERE { wd:Q95 wdt:P1128 ?uri . }
answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '1'}}]
get answer: 
 [{'c': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '1'}}]


In [363]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 87
question: 
Which book has the most pages?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31/wdt:P279* wd:Q571 . ?uri wdt:P1104 ?number . } ORDER BY DESC(?number) LIMIT 1
get sparql:  
SELECT DISTINCT ?s1 WHERE { ?s1 <http://www.wikidata.org/prop/direct/P106> <http://www.wikidata.org/entity/Q49757> . ?s2 <http://www.wikidata.org/prop/direct/P50> ?s1 . ?s2 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q571> . } GROUP BY ?s1 ORDER BY
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15712417'}}]
get answer: 
 []


In [364]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 148
question: 
What is the largest state in the United States?
sparql: 
PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P2046 ?area . } ORDER BY DESC(?area) LIMIT 1
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?res WHERE { ?res wdt:P17 wd:Q30 . ?res wdt:P31/wdt:P279* wd:Q515 . ?res wdt:P1082 ?population . } ORDER BY DESC(?population) LIMIT 1
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q797'}}]
get answer: 
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q60'}}]


In [365]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 43
question: 
Give me the websites of companies with more than 500000 employees.
sparql: 
SELECT DISTINCT ?string WHERE { ?uri <http://www.wikidata.org/prop/direct/P31>/(<http://www.wikidata.org/prop/direct/P279>*) <http://www.wikidata.org/entity/Q4830453> . ?uri <http://www.wikidata.org/prop/direct/P1128> ?number . FILTER(?number > 500000) . ?uri <http://www.wikidata.org/prop/direct/P856> ?string . }
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?type wdt:P279* wd:Q4830453 . ?uri wdt:P31 ?type ; wdt:P1128 ?num . FILTER(?num > 1000000) }
answer: 
 [{'string': {'type': 'uri', 'value': 'https://www.amazon.com'}}, {'string': {'type': 'uri', 'value': 'http://www.rzd.ru/'}}, {'string': {'type': 'uri', 'value': 'http://www.avic.com.cn'}}, {'string': {'type': 'uri', 'value': 'http://www.cnpc.com.cn/'}}, {'string': {'type': 'uri', 'value': 'http://www.abchina.com/'}}, {'string': {'type': 'uri', 'va

In [366]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

id: 179
question: 
What were the names of the three ships by Columbus?
sparql: 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P31/wdt:P279* wd:Q11446 . ?uri wdt:P1344/wdt:P710 wd:Q7322 . } 
get sparql:  
PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?uri wdt:P1427 wd:Q177477 }
answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q107900'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q501355'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q501652'}}]
get answer: 
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q144502'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q182524'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q192372'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2

# Summary

- Some SPARQL learned from training set
- most of predicted SPARQL are more or less related to the question, even the predicted answer is direct from training set
- loss the ability of generalization 
- Some questions in test set also appear in training set, but with different SPARQL
- fine-tuned model can write SPARQL, but most of them does not give any answer

## Out-of-Scope questions

99, 98, 84, 81, 66, 
64, 56, 49, 44, 42, 
4, 29, 68, 22, 214, 
210, 199, 203, 196, 187,
176, 62, 173, 168, 166,
160, 157, 150, 143, 140,
139, 136, 132, 128, 124,
111, 117, 59, 104, 10,
1, 178, 129, 181, 7,
135, 50, 71, 213, 105,
102, 145, 162, 212, 96,
40, 80, 159, 107, 163,
51, 144, 131, 120, 164,
63, 27, 207, 24, 133,
103, 108, 15, 82, 201,
171, 182, 114, 194, 175,
60, 110, 206, 38, 192,
78, 190

## In-Scope questions

86, 32, 31, 197, 188,
149, 25, 137, 122, 113,
183, 209, 21, 154, 198,
211, 156, 165, 9, 13, 
97, 19

## In training set

158, 39, 52, 141, 151,
45, 92, 169